In [8]:
from PyPDF2 import PdfFileReader
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import sys
import os


def pdf2text(pdf):
    #'''Iterate over pages and extract text'''
    text = ''
    for i in range(pdf.getNumPages()):
        page = pdf.getPage(i)
        text = text + page.extractText()
    return text

def stem_tokenize(document):
    #'''return stemmed words longer than 2 chars and all alpha'''
    tokens = [stem(w) for w in document.split() if len(w) > 2 and w.isalpha()]
    return tokens

def tokenize(document):
    #'''return words longer than 2 chars and all alpha'''
    tokens = [w for w in document.split() if len(w) > 2 and w.isalpha()]
    return tokens

def build_corpus_from_dir(dir_path):
    corpus = []
    for root, dirs, filenames in os.walk(dir_path):
        for name in filenames:
            f  = os.path.join(root, name)
            pdf = PdfFileReader(f, "rb")
            document = pdf2text(pdf)
            corpus.append(document)
    return corpus

In [9]:
corpus = build_corpus_from_dir(sys.argv[1])[8:]
vectorizer = TfidfVectorizer(tokenizer = tokenize, stop_words = None)
vectorizer.fit(corpus)
indices = np.argsort(vectorizer.idf_)[::-1]
features = vectorizer.get_feature_names()
for doc in corpus:
    print(' '.join(doc.split())[500])
    tdm = vectorizer.transform([doc])
    dense = tdm.todense()
    episode = dense[0].tolist()[0]
    phrase_scores = [pair for pair in zip(range(0, len(episode)), episode) if pair[1] > 0]
    sorted_phrase_scores = sorted(phrase_scores, key=lambda t: t[1] * -1)
    for phrase, score in [(features[word_id], score) for (word_id, score) in sorted_phrase_scores][:10]:
        print('{0: <20} {1}'.format(phrase, score))

ValueError: empty vocabulary; perhaps the documents only contain stop words